# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 10 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.

### Exercise 10.1 

**Use QuTiP to verify that the two operations `ApplyCNOT` and `ApplyCNOTTheOtherWay` can be simulated by the same unitary matrix, and thus do the exact same thing.**

This first snippet shows the unitary matrix for `ApplyCNOT` which is equvalent to the QuTiP function `cnot`.

In [ ]:
from qutip.qip.operations import cnot

cnot()

The matrix above is the same as the one below which represents surrounding a `CNOT` operation with `H` on all qubits, and flipping the control and target qubits.

In [ ]:
from qutip.tensor import tensor
from qutip.qip.operations import hadamard_transform

(
      tensor([hadamard_transform(), hadamard_transform()])
    * cnot(None,1,0)
    * tensor([hadamard_transform(), hadamard_transform()])
)

----
### Exercise 10.2

**Just as you can use three classical `XOR` instructions to implement an in-place classical swap, you can use three `CNOT` operations to do the same thing as a single `SWAP` operation.
In fact, the following Q# snippet does the same thing as `SWAP(left, right)`:**

```
CNOT(left, right);
CNOT(right, left);
CNOT(left, right);
```
**Double-check that this is the same as `SWAP(left, right)`, both by using `AssertOperationsEqualReferenced`, and by using QuTiP.**

In [ ]:
import qsharp

In [ ]:
qsharp.compile("""
    operation SwapWithCnot(pair : Qubit[]) : Unit {
        CNOT(pair[0], pair[1]);
        CNOT(pair[1], pair[0]);
        CNOT(pair[0], pair[1]);
    }

    operation SwapDirectly(pair : Qubit[]) : Unit is Adj {
        SWAP(pair[0], pair[1]);
    }
""")

In [ ]:
assert_swap = qsharp.compile("""
    open Microsoft.Quantum.Diagnostics;
    
    operation AssertSwapEqualsThreeCnot() : Unit {
        AssertOperationsEqualReferenced(2, SwapWithCnot, SwapDirectly);
    }
""")

In [ ]:
assert_swap.simulate()



> **Extra credit**: `SWAP(left, right)` is the same as `SWAP(right, left)`, so the snippet above should work even if you start with `CNOT(right, left)` instead. Double-check that!

In [ ]:
qsharp.compile("""
    operation ReverseSwapWithCnot(pair : Qubit[]) : Unit{
        CNOT(pair[1], pair[0]);
        CNOT(pair[0], pair[1]);
        CNOT(pair[1], pair[0]);
    }

    operation ReverseSwapDirectly(pair : Qubit[]) : Unit is Adj {
        SWAP(pair[1], pair[0]);
    }
""")

assert_swap_reverse = qsharp.compile("""
    open Microsoft.Quantum.Diagnostics;
    
    operation AssertSwapEqualsThreeCnot() : Unit {
        AssertOperationsEqualReferenced(2, ReverseSwapWithCnot, ReverseSwapDirectly);
    }
""")

In [ ]:
assert_swap_reverse.simulate()

----
### Exercise 10.3

**Using QuTiP, check that when run on two-qubit registers, the two programs from the listing below can be simulated by the same unitary matrix and thus do the same thing to their input registers.**

```
open Microsoft.Quantum.Diagnostics;

operation ApplyXUsingCNOTs(register : Qubit[])
: Unit is Adj + Ctl {
    within {
        ApplyToEachCA(
            CNOT(register[0], _),
            register[1...]
        );
    } apply {
        X(register[0]);
    }
}

operation CheckThatThisWorks() : Unit {
    AssertOperationsEqualReferenced(2,
        ApplyXUsingCNOTs,
        ApplyToEachCA(X, _)
    );
    Message("Woohoo!");
}
```

In [ ]:
from qutip import sigmax, qeye
from qutip.tensor import tensor
from qutip.qip.operations import cnot
from functools import reduce 

def apply_x_using_cnot(n : int):
    within = reduce((lambda x, y: y * x), [cnot(n, 0, i) for i in range(1, n)])  
    return within.dag() * tensor([sigmax()] + [qeye(2)] * (n - 1)) * within

def apply_to_each_x(n : int):
    return tensor([sigmax()] * n)

print(apply_x_using_cnot(3))
print(apply_to_each_x(3))

You can see that the above two matricies are the same and thus represent the same operation.

----
### Exercise 10.4

**Try modifying the listing from exercise 10.3 to see if both programs are equivalent when applied to more than two qubits.**

> **NOTE:** It can be pretty expensive to use `AssertOperationsEqualReferenced` for more than a few qubits.

In [ ]:
[_, check_three_qubit, check_eight_qubit] = qsharp.compile("""
open Microsoft.Quantum.Diagnostics;

operation ApplyXUsingCNOTs(register : Qubit[]) : Unit is Adj + Ctl {
    within {
        ApplyToEachCA(
            CNOT(register[0], _),
            register[1...]
        );
    } apply {
        X(register[0]);
    }
}

operation CheckThatThisWorks3() : Unit {
    AssertOperationsEqualReferenced(3,
        ApplyXUsingCNOTs,
        ApplyToEachCA(X, _)
    );
    Message("Woohoo!");
}

operation CheckThatThisWorks8() : Unit {
    AssertOperationsEqualReferenced(8,
        ApplyXUsingCNOTs,
        ApplyToEachCA(X, _)
    );
    Message("Woohoo!");
}

""")

In [ ]:
check_three_qubit.simulate()
check_eight_qubit.simulate()

For at least a small sample, the assert succedes and so you know the two programs are the same. It turns out that they will always be the same, no matter the number of qubits used.

----
### Exercise 10.5


**Try preparing your register in states other than $\left|00\right\rangle$ before calling `ApplyRotationAboutXX`.
Does your operation do what you expected?**

> **HINT:** Recall from Part I that you can prepare a copy of the $\left|1\right\rangle$ state by applying an `X` operation, and that you can prepare $\left|+\right\rangle$ by applying an `H` operation.

In [ ]:
qsharp.compile("""
    operation ApplyRotationAboutXX(
        angle : Double, register : Qubit[]
    ) : Unit is Adj + Ctl {
        within {
            CNOT(register[0], register[1]);
        } apply {
            Rx(angle, register[0]);
        }
    }
""")

rotate_zeros_about_xx = qsharp.compile("""
    open Microsoft.Quantum.Diagnostics;

    operation RotateZeroAboutXX(angle : Double) : Unit {
        use register = Qubit[2];
        ApplyRotationAboutXX(angle, register);
        DumpMachine();
        Message("\n");
        ResetAll(register);
    }
""")
    
rotate_plus_about_xx = qsharp.compile("""
    open Microsoft.Quantum.Diagnostics;

    operation RotatePlusAboutXX(angle : Double) : Unit {
        use register = Qubit[2];
        ApplyToEachCA(H, register);
        ApplyRotationAboutXX(angle, register);
        DumpMachine();
        Message("\n");
        ResetAll(register);
    }
""")
    
rotate_ones_about_xx = qsharp.compile("""
    open Microsoft.Quantum.Diagnostics;

    operation RotateOnesAboutXX(angle : Double) : Unit {
        use register = Qubit[2];
        ApplyToEachCA(X, register);
        ApplyRotationAboutXX(angle, register);
        DumpMachine();
        Message("\n");
        ResetAll(register);
    }
""")

With that Q# code compiled, make a table of the resulting states for a range of angles from 0 to $2\pi$. Here we started with the $|00\rangle$ state for comparison.

In [ ]:
import numpy as np
[rotate_zeros_about_xx.simulate(angle=a * np.pi / 4) for a in range(8)]

Now repeat the rotation angles but start with the state $|++\rangle$.

In [ ]:
import numpy as np
[rotate_plus_about_xx.simulate(angle=a * np.pi / 4) for a in range(0, 8)]

Note that these are all equivalent up to a global phase; rotating the |++⟩ state around the 𝑋𝑋-axis doesn't do anything.

One more time, repeat the angles with the inital state of $|11\rangle$.

In [ ]:
import numpy as np
[rotate_ones_about_xx.simulate(angle=a * np.pi / 4) for a in range(0, 8)]

----
### Exercise 10.6

**Try using `DumpMachine` to explore how the `Rx` operation acts on a single qubit, and compare to the two-qubit rotation about the $X \otimes X$ axis that you implemented in the snippet below.
How are the two rotation operations similar, and how do they differ?**
```
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation ApplyRotationAboutXX(angle : Double, register : Qubit[])
: Unit is Adj + Ctl {
    within {
        CNOT(register[0], register[1]);
    } apply {
        Rx(angle, register[0]);
    }
}

operation DumpXXRotation() : Unit {
    let angle = PI() / 2.0;
    use register = Qubit[2];
    ApplyRotationAboutXX(angle, register);
    DumpMachine();
    ResetAll(register);
}
```

In [ ]:
[_, dump_rx_rotation, dump_xx_rotation] = qsharp.compile("""
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Math;

    operation ApplyRotationAboutXX(
        angle : Double, register : Qubit[]
    ) : Unit is Adj + Ctl {
        within {
            CNOT(register[0], register[1]);
        } apply {
            Rx(angle, register[0]);
        }
    }

    operation DumpRxRotation(angle : Double) : Unit {
        use q = Qubit();
        Rx(angle, q);
        DumpMachine();
        Message("\n");
        Reset(q);
    }

    operation DumpXXRotation(angle : Double) : Unit {
        use register = Qubit[2];
        ApplyRotationAboutXX(angle, register);
        DumpMachine();
        Message("\n");
        ResetAll(register);
    }
""")

In [ ]:
import numpy as np
[dump_rx_rotation.simulate(angle=a * np.pi / 4) for a in range(0, 8)]

In [ ]:
[dump_xx_rotation.simulate(angle=a * np.pi / 4) for a in range(0, 8)]

The rotations are similar in that the amplitudes for the first ($|0\rangle$ or $|00\rangle$) and last state ($|1\rangle$ or $|11\rangle$) have the same amplitudes. They are obviously different from the standpoint they operate on different numbers of qubits.

**Compare rotating about the $X \otimes X$ axis with applying an `Rx` operation to each qubit in a two-qubit register.**

In [ ]:
dump_rxrx_rotation = qsharp.compile("""
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Math;

    operation DumpRxRxRotation(angle : Double) : Unit {
        use register = Qubit[2];
        ApplyToEach(Rx(angle, _), register);
        DumpMachine();
        Message("\n");
        ResetAll(register);
    }
""")

In [ ]:
[dump_rxrx_rotation.simulate(angle=a * np.pi / 4) for a in range(0, 8)]

You can see here that emphatically applying the `Rx` operation to each of the two qubits in a register is _not_ the same as rotating about the $XX$-axis.

----
### Epilogue

_The following cell logs what version of the components this was last tested with._

In [1]:
qsharp.component_versions()

Preparing Q# environment...


{'iqsharp': LooseVersion ('0.17.2105143879'),
 'Jupyter Core': LooseVersion ('1.5.0.0'),
 '.NET Runtime': LooseVersion ('.NETCoreApp,Version=v3.1'),
 'qsharp': LooseVersion ('0.17.2105.143879')}